# <center> Image Manipulation

### Load Libraries

In [0]:
from tensorflow.keras.applications.vgg16  import VGG16
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input, InputLayer, Conv2D,UpSampling2D , Flatten,MaxPooling2D,Conv2DTranspose
from tensorflow.keras.models import Model,Sequential
from keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from PIL import Image
import random
from math import ceil
import imageio
import cv2
from skimage.data import astronaut
from skimage.filters import gaussian,sobel
from skimage import transform,io

##### Set file path

In [0]:
os.chdir('D:\\Proj\\vinita\\Project 3- Image Manipulation\\manipulation_data\\Real_data\\')

### Data Preperation

###### Sort and Collect Authentic Data

In [0]:
rootdir = os.getcwd()
filelist=[]  
tampered_data=[]    
real_data=[]        
i=0                
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = subdir + os.sep + file
        if (filepath.find('gt') >0):
            i=i+1
            print(filepath)
            width = 300
            height = 300
            image1 = cv2.imread(filepath,0)
            transformed_image = transform.resize(image1, (300,300), mode='symmetric', preserve_range=True)
            transformed_image=(transformed_image-transformed_image.mean())/255
            real_data.append(transformed_image)

##### Sort and Collect Doctored Data

In [0]:
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = subdir + os.sep + file

        if (filepath.find('gt') > 0):
            i=i+1
            print(filepath)
            im = Image.open(filepath) 
            width = 300
            height = 300
            image1 = cv2.imread(filepath,0)
            transformed_image = transform.resize(image1, (300,300), mode='symmetric', preserve_range=True)
            transformed_image=(transformed_image-transformed_image.mean())/255
            tampered_data.append(transformed_image)

##### Transform and Convert Data

In [0]:
transformed_image=(transformed_image-transformed_image.mean())/255

transformed_image.max()
transformed_image.min()

real_data=np.asarray(real_data)
tampered_data=np.asarray(tampered_data)

##### Create Combined Dataset

In [0]:
combined_input=np.concatenate([real_data, tampered_data])

y_combined=np.zeros(real_data.shape[0]+tampered_data.shape[0])
y_combined[:real_data.shape[0]]=1

combined_input=combined_input.reshape(624,300,300,1)

trainsize= ceil(0.8 * combined_input.shape[0])
testsize= ceil(0.2 * combined_input.shape[0])+1


trainsel=np.random.randint(low=0,high=combined_input.shape[0],size=trainsize)
testsel=np.random.randint(low=0,high=combined_input.shape[0],size=testsize)
train_inp=combined_input[trainsel,]
test_inp=combined_input[testsel,]

train_out=y_combined[trainsel,]
test_out=y_combined[testsel,]

##### Define Optomizer

In [0]:
def adam_optimizer():
    return Adam(lr=0.001,beta_1=0.9)

### Structure Model

In [0]:
model = Sequential()
model.add(InputLayer(input_shape=(300, 300, 1)))
model.add(Conv2D(32, (3, 3), activation='tanh', padding='same', strides=2))

model.add(Conv2D(32, (3, 3), activation='tanh', padding='same'))
model.add(layers.LeakyReLU(0.6))
model.add(layers.Dropout(0.4))

model.add(Conv2D(32, (3, 3), activation='tanh', padding='same', strides=2))
model.add(layers.LeakyReLU(0.3))
model.add(layers.Dropout(0.2))

model.add(Conv2D(32, (3, 3), activation='tanh', padding='same', strides=2))
model.add(layers.LeakyReLU(0.3))
model.add(layers.Dropout(0.2))

model.add(Conv2D(32, (3, 3), activation='tanh', padding='same', strides=2))
model.add(layers.LeakyReLU(0.3))
model.add(layers.Dropout(0.2))

model.add(Conv2D(1, (3, 3), activation='tanh', padding='same',strides=2))
model.add(MaxPooling2D(pool_size = (3, 3)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
print(model.summary())

### Train Model

In [0]:
model.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
model.fit(x=train_inp, y=train_out, batch_size=100,epochs=3)

### Test Model

In [0]:
train_pred = model.predict(train_inp)
test_pred = model.predict(test_inp)

### Check Results

In [0]:
check=np.interp(train_pred, (train_pred.min(), train_pred.max()), (0,1))
check1=np.interp(test_pred, (test_pred.min(), test_pred.max()), (0,1))

plt.hist(check)
plt.hist(check1)

train_check = np.concatenate((train_out.reshape(-1,1),check.reshape(-1,1)),axis=1)
test_check = np.concatenate((test_out.reshape(-1,1),check1.reshape(-1,1)),axis=1)